In [1]:
import cv2
import numpy as np

def ORB_detector(new_image, image_template):
    # Function that compares input image to template
    # It then returns the number of ORB matches between them
    image1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Create ORB detector with 1000 keypoints with a scaling pyramid factor of 1.2
    orb = cv2.ORB_create(1000, 1)

    # Detect keypoints of original image
    (kp1, des1) = orb.detectAndCompute(image1, None)

    # Detect keypoints of rotated image
    (kp2, des2) = orb.detectAndCompute(image_template, None)

    # Create matcher 
    # Note we're no longer using Flannbased matching
    bf = cv2.BFMatcher()
    # Do matching
    matches = bf.match(des1,des2)

    # Sort the matches based on distance.  Least distance
    # is better
    #matches = sorted(matches, key=lambda val: val.distance)
    return len(matches)

In [2]:
cap = cv2.VideoCapture('sm.mp4')

# Load our image template, this is our reference image
image_template = cv2.imread('sm_lecture.png', 0)
#image_template = cv2.imread('images/kitkat.jpg', 0) 

while True:
    # Get webcam images
    ret, frame = cap.read()
    if ret:
        # Get height and width of webcam frame
        height, width = frame.shape[:2]

        # Define ROI Box Dimensions (Note some of these things should be outside the loop)
        top_left_x = int(width / 3)
        top_left_y = int((height / 2) + (height / 4))
        bottom_right_x = int((width / 3) * 2)
        bottom_right_y = int((height / 2) - (height / 4))

        # Draw rectangular window for our region of interest
        cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), 255, 3)

        # Crop window of observation we defined above
        cropped = frame[bottom_right_y:top_left_y , top_left_x:bottom_right_x]

        # Flip frame orientation horizontally
        frame = cv2.flip(frame,1)

        # Get number of ORB matches 
        matches = ORB_detector(cropped, image_template)

        # Display status string showing the current no. of matches 
        output_string = "Matches = " + str(matches)
        cv2.putText(frame, output_string, (50,450), cv2.FONT_HERSHEY_COMPLEX, 2, (250,0,150), 2)

        # Our threshold to indicate object deteciton
        # For new images or lightening conditions you may need to experiment a bit 
        # Note: The ORB detector to get the top 1000 matches, 350 is essentially a min 35% match
        threshold = 10

        # If matches exceed our threshold then object has been detected
        if matches > threshold:
            cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,255,0), 3)
            cv2.putText(frame,'Object Found',(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)

        cv2.imshow('Object Detector using ORB', frame)
        if cv2.waitKey(1) == 13: #13 is the Enter Key
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2
  
     
# Read the query image as query_img
# and train image This query image
# is what you need to find in train image
# Save it in the same directory
# with the name image.jpg 
query_img = cv2.imread('query.jpg')
train_img = cv2.imread('train.jpg')
  
# Convert it to grayscale
query_img_bw = cv2.cvtColor(query_img,cv2.COLOR_BGR2GRAY)
train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)
  
# Initialize the ORB detector algorithm
orb = cv2.ORB_create()
  
# Now detect the keypoints and compute
# the descriptors for the query image
# and train image
queryKeypoints, queryDescriptors = orb.detectAndCompute(query_img_bw,None)
trainKeypoints, trainDescriptors = orb.detectAndCompute(train_img_bw,None)
 
# Initialize the Matcher for matching
# the keypoints and then match the
# keypoints
matcher = cv2.BFMatcher()
matches = matcher.match(queryDescriptors,trainDescriptors)
  
# draw the matches to the final image
# containing both the images the drawMatches()
# function takes both images and keypoints
# and outputs the matched query image with
# its train image
final_img = cv2.drawMatches(query_img, queryKeypoints,
train_img, trainKeypoints, matches[:20],None)
  
final_img = cv2.resize(final_img, (1000,650))
 
# Show the final image
cv2.imshow("Matches", final_img)
cv2.waitKey(3000)